# ts2g using ordinal partition strategy

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import os
import csv
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from core import model 
from timeseries.ordinal_partition import TimeseriesToOrdinalPatternGraph
# from timeseries.strategies import TimeseriesToGraphStrategy

## Loading data 

We first load the dataset :)

In [3]:
amazon_data = pd.read_csv(os.path.join(os.getcwd(), "amazon", "AMZN.csv"))   

To properly use the ‘Date’ column, we convert it to datetime format and ensure the dates are recognized as date objects. We then set the ‘Date’ column as the index of the DataFrame. This creates a time-series structure, facilitating analysis and visualization over time.

In [4]:
amazon_data["Date"] = pd.to_datetime(amazon_data["Date"])
amazon_data.set_index("Date", inplace=True)

# How does the time series look like?

In [5]:
def plot_timeseries(sequence, title, x_legend, y_legend, color):
    plt.figure(figsize=(10, 6))
    plt.plot(sequence, linestyle='-', color=color)
    
    plt.title(title)
    plt.xlabel(x_legend)
    plt.ylabel(y_legend)
    plt.grid(True)
    plt.show()

In [6]:
def plot_timeseries_sequence(df_column, title, x_legend, y_legend, color='black'):
    sequence = model.Timeseries(model.TimeseriesArrayStream(df_column)).to_sequence()
    plot_timeseries(sequence, title, x_legend, y_legend, color)

In [7]:
plot_timeseries_sequence(amazon_data["Close"], "Original Sequence", "Year", "Value")

Given the great length of the timeseries, let's focus on some sub-segments, so that we can better appreciate its behavior.

In [8]:
segment_1 = amazon_data[60:120]
segment_2 = amazon_data[4000:4060]
segment_3 = amazon_data[6000:6060]

How do the plots for these segments look like?

In [9]:
plot_timeseries_sequence(segment_1["Close"], "Segment 1", "Year", "Value", 'gray')

In [10]:
plot_timeseries_sequence(segment_2["Close"], "Segment 2", "Year", "Value", 'green')

In [11]:
plot_timeseries_sequence(segment_3["Close"], "Segment 3", "Year", "Value", 'blue')

Let's turn the timeseries into networks (graphs)!

In [25]:
def sequence_to_graph(time_series, w, tau,use_quantiles = True, Q=4, color= 'gray'):
    strategy = TimeseriesToOrdinalPatternGraph (w, tau, use_quantiles = use_quantiles, Q= Q)
    g = strategy.to_graph(time_series)
    pos= nx.circular_layout(g)
    plt.figure(1,figsize=(8,8))
    
    all_weights = []
    for (q1, q2, data) in g.edges(data=True):
        all_weights.append(data['weight'])
    unique_weights = list(set(all_weights))
    
    nx.draw_networkx_nodes(g, pos, g.nodes, node_size=300, node_color = color)
    
    labels = {node: node for node in g.nodes()}
    nx.draw_networkx_labels(g, pos, labels, font_size=12, font_color='black')
    
    for weight in unique_weights:
        weighted_edges = [(q1, q2) for (q1, q2, edge_attr) in g.edges(data=True) if edge_attr['weight']==weight]
        width = weight*len(g.nodes)*5.0/sum(all_weights)
        nx.draw_networkx_edges(g,pos,edgelist=weighted_edges,width=width,edge_color='black',node_size=300)
        
plt.show()

In [26]:
sequence_to_graph(segment_1["Close"],3,1,use_quantiles = False,color='gray')

In [30]:
sequence_to_graph(segment_1["Close"],3,1,use_quantiles = True, Q = 4,color='gray')

In [31]:
sequence_to_graph(segment_2["Close"],3,1,use_quantiles = False,color= 'green')

In [41]:
sequence_to_graph(segment_2["Close"],3,1,use_quantiles = True,Q=4, color= 'green')

In [42]:
sequence_to_graph(segment_3["Close"],3,1,use_quantiles = False,color= 'blue')

In [43]:
sequence_to_graph(segment_3["Close"],3,1,use_quantiles = True, Q=4,color= 'blue')

In [31]:
# x = list(range(1,21))
x = list(range(1,21))
y = [1,2,3,4,5,6,7,8,9,10,10, 9,8,7,6,5,4,3,2,1]

# y = [0.35, 0.9, 0.3, 0.8, 0.6, 0.95, 0.1, 0.4, 0.9, 0.15, 0.5, 5, 0.05, 0.1, 0.2, 0.7, 0.6, 0.9, 0.3, 0.4]

df = pd.DataFrame({'x': x, 'y': y})
df.set_index("x", inplace=True)

In [32]:
sequence_to_graph(df["y"],3,1,use_quantiles = False,color='gray')

In [33]:
sequence_to_graph(df["y"],3,1,use_quantiles = True,color='gray')